In [1]:
import nltk
import pandas as pd
import numpy as np
import pickle

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

### Load Data
#### Load "derecha diario" dataset from csv.

In [2]:
df = pd.read_csv('./data/derecha_econ_news.csv', index_col=0)

In [3]:
df.shape

(480, 6)

In [4]:
df.head(5)

,date,flyer,title,lead,body,url
0,17/03/2021,Economía,Argentina y España lideran los peores resultad...,La coalición social-comunista española y el ki...,Argentina y España se posicionan como los país...,https://derechadiario.com.ar/economia/argentin...
1,17/03/2021,Economía,Bomba de las Leliq: el Banco Central tuvo que ...,La política de esterilización que emprendió el...,Después de un 2020 marcado por la emisión mone...,https://derechadiario.com.ar/economia/bomba-de...
2,17/03/2021,Economía,Estados Unidos: en febrero se desaceleró el cr...,En medio de tormentas invernales y subas en lo...,La actividad del sector servicios en los Estad...,https://derechadiario.com.ar/economia/estados-...
3,17/03/2021,Economía,La producción industrial de Estados Unidos cay...,Las medidas energéticas del Gobierno demócrata...,La Reserva Federal informó sobre una dura cont...,https://derechadiario.com.ar/economia/la-produ...
4,16/03/2021,Economía,Fracaso del sistema de jubilación estatal: un ...,El sistema de pensiones argentino no es capaz ...,La estatización del sistema privado AFJP y el ...,https://derechadiario.com.ar/economia/fracaso-...


In [5]:
# remove unicode
df.body.replace('\n', ' ', regex=True ,inplace=True)
df.body.replace('\xa0', ' ', regex=True ,inplace=True)

### Topics
Selection of topics from news titles of the portal's economic section.

In [6]:
inflation = df.loc[df.title.str.contains('inflacion|inflación|emisión|salarios|salario|indec', na=False)]
inflation.shape

(64, 6)

In [7]:
exchange = df.loc[df.title.str.contains('dolar|dólar|dolares|dólares|dolarización|cambio|cepo', na=False)]
exchange.shape

(40, 6)

### Corpus (Body)
Clipping of the selected news body from 2020 and erasing nulls

#### Topic: Inflation

In [8]:
year = inflation.date.str.contains('2020', na=False)
year.sum()

45

In [9]:
inflation_2020 = inflation[year]
inflation_2020.dropna(inplace=True)
inflation_2020.shape

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(45, 6)

In [10]:
inflation_2020 = inflation_2020.reset_index()

In [11]:
inflation_2020.insert(loc=1, column='topics', value='inflation')

In [12]:
inflation_2020.head(5)

,index,topics,date,flyer,title,lead,body,url
0,110,inflation,24/12/2020,Economía,Represión inflacionaria y riesgo para 2021: Mo...,La famosa multinacional financiera alerta sobr...,En medio de una época típicamente marcada por ...,https://derechadiario.com.ar/economia/represio...
1,121,inflation,16/12/2020,Economía,A pesar de las medidas de último recurso de Gu...,El INDEC difundió los resultados del índice de...,El Instituto Nacional de Estadística y Censos ...,https://derechadiario.com.ar/economia/a-pesar-...
2,122,inflation,15/12/2020,Economía,La AFIP no da respiro: el mínimo no imponible ...,"El Mínimo No Imponible sólo aumentará 35,38% y...","Desde 2018, según lo establecido por la Ley 27...",https://derechadiario.com.ar/economia/la-afip-...
3,124,inflation,14/12/2020,Economía,Fracaso de los Precios Máximos: 580 negocios f...,Pese a los rigurosos controles de la Secretarí...,El Índice de Precios al Consumidor y las expec...,https://derechadiario.com.ar/economia/fracaso-...
4,126,inflation,12/12/2020,Economía,Crece el empleo estatal mientras cae el sector...,"Pese al cepo laboral y las restricciones, el s...","Según el INDEC, el sector privado formal habrí...",https://derechadiario.com.ar/economia/crece-el...


#### Topic: Currency Exchange

In [13]:
year = exchange.date.str.contains('2020', na=False)
year.sum()

33

In [14]:
exchange_2020 = exchange[df.date.str.contains('2020', na=False)]
exchange_2020.dropna(inplace=True)
exchange_2020.shape

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(33, 6)

In [15]:
exchange_2020 = exchange_2020.reset_index()

In [16]:
exchange_2020.insert(loc=1, column='topics', value='exchange')

In [17]:
exchange_2020.head(5)

,index,topics,date,flyer,title,lead,body,url
0,102,exchange,30/12/2020,Economía,El Banco Central Argentino va a restringir las...,La autoridad monetaria argentina seguirá restr...,El tipo de cambio en los mercados oficiales se...,https://derechadiario.com.ar/economia/el-banco...
1,130,exchange,09/12/2020,Economía,El reconocido ejecutivo Mark Mobius usó el fra...,El famoso hombre de las finanzas internacional...,"Mark Mobius, el reconocido gestor de mercados ...",https://derechadiario.com.ar/economia/el-recon...
2,149,exchange,24/11/2020,Economía,El economista Steve Hanke advierte que la infl...,El economista norteamericano de la Universidad...,"El economista Steve Hanke, profesor e investig...",https://derechadiario.com.ar/economia/el-econo...
3,154,exchange,18/11/2020,Economía,Desesperación en el régimen chavista: el Banco...,"Tras casi cinco años de hiperinflación, el Ban...","Según un reciente informe de Bloomberg, fuente...",https://derechadiario.com.ar/economia/desesper...
4,159,exchange,13/11/2020,Economía,Nueva subida del dólar blue: ya acumula $23 en...,"Tras haber caído por debajo de los $150, el dó...",Hoy el dólar libre llegó a venderse a $174 en ...,https://derechadiario.com.ar/economia/nueva-su...


In [18]:
data_clean = inflation_2020.append(exchange_2020)
data_clean.shape

(78, 8)

I drop those news labeled under both topics

In [19]:
data_clean = data_clean.drop_duplicates(subset='index',keep=False)
data_clean.shape

(72, 8)

Check the number of words per article's body and discard those incompleted

In [20]:
filter = data_clean.body.apply(lambda x: len(word_tokenize(x))) < 150
filter.sum()

0

In [21]:
data_clean.drop(data_clean[filter].index, axis=0, inplace=True)

#### Save selection

In [22]:
data_clean.reset_index(drop=True, inplace=True)

In [23]:
data_clean.drop('index', axis=1, inplace=True)

In [24]:
print('num inflation news:', data_clean.loc[data_clean.topics == 'inflation',].shape[0])
print('num exchange news:', data_clean.loc[data_clean.topics == 'exchange',].shape[0])

num inflation news: 42
num exchange news: 30


In [25]:
data_clean.sample(5)

,topics,date,flyer,title,lead,body,url
59,exchange,11/06/2020,Economía,El Banco Central dispone una pequeña flexibili...,"El titular del BCRA Miguel Pesce, dispuso una ...",En un golpe de confianza producido por la recu...,https://derechadiario.com.ar/economia/el-banco...
18,inflation,11/09/2020,Economía,Presupuesto 2021: el Gobierno estima una recup...,El ministerio de Economía prepara su ley de pr...,El Gobierno está preparando los primeros borra...,https://derechadiario.com.ar/economia/presupue...
0,inflation,24/12/2020,Economía,Represión inflacionaria y riesgo para 2021: Mo...,La famosa multinacional financiera alerta sobr...,En medio de una época típicamente marcada por ...,https://derechadiario.com.ar/economia/represio...
2,inflation,15/12/2020,Economía,La AFIP no da respiro: el mínimo no imponible ...,"El Mínimo No Imponible sólo aumentará 35,38% y...","Desde 2018, según lo establecido por la Ley 27...",https://derechadiario.com.ar/economia/la-afip-...
71,exchange,23/02/2020,Economía,Los argentinos quieren un cambio en su moneda,De acuerdo al estudio “Cryptocurrencies and th...,Los ciudadanos de países con poca confianza en...,https://derechadiario.com.ar/economia/los-arge...


In [26]:
pickle.dump(data_clean, open( "data/df_clean_der.pkl", "wb" ))

In [27]:
corpus = data_clean
corpus.shape

(72, 7)

### Tokenization
#### Manual

In [28]:
import re
import unidecode

"Stop words", special characters, accents and numbers are removed

In [29]:
stopwords_sp = stopwords.words('spanish')

In [30]:
def tokenizer(texto):

    alphanumeric = re.sub(r'([^\s\w]|_)+', '', texto).lower()
    no_accents = unidecode.unidecode(alphanumeric)
    
    tockens = word_tokenize(no_accents)
    
    tockens_clean = [tocken for tocken in tockens if tocken not in stopwords_sp and tocken.isalpha()]
    
    terminos = tockens_clean

    return terminos

In [31]:
tokenized = [tokenizer(documento) for documento in corpus.body]

In [32]:
vocabulario = sorted(list(set([word for group in tokenized for word in group])))

In [33]:
len(vocabulario)

5373

#### CountVectorizer

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

Removal of special characters function to load in vectorizer

In [35]:
def removal(text):
    text = re.sub(r'(\d|\$|\%|\+)', '', text.lower())
    return re.sub(r'\d+', '', text)

In [36]:
vectorizer = CountVectorizer(stop_words = stopwords_sp, lowercase = True, strip_accents='unicode', preprocessor=removal)

In [37]:
vectorizer.fit(corpus.body)

CountVectorizer(preprocessor=<function removal at 0x7fb6dcb33290>,
                stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...],
                strip_accents='unicode')

In [38]:
len(vectorizer.vocabulary_), len(vectorizer.get_feature_names())

(5233, 5233)

I prefare the manual tokenization results since the process is clearer and there are more terms.

### Pickle

Manual tokens

In [39]:
def token_matrix_maker(data, vocabulario):
    matriz = np.zeros(shape = (len(data), len(vocabulario)), dtype='int')
    for i, documento in enumerate(data):
        for termino in documento:
            matriz[i, vocabulario.index(termino)] += 1
    return matriz

In [40]:
matrix = token_matrix_maker(tokenized, vocabulario)

In [41]:
# make dataframe
token_df = pd.DataFrame(matrix, columns=vocabulario, index = ['doc' + str(i + 1) for i in range(len(tokenized))] )

In [42]:
# add classification column
topics = ['inflation' for i in range((corpus.topics == 'inflation').sum())] + ['exchange' for i in range((corpus.topics == 'exchange').sum())]
token_df.insert(loc=0, column='topics', value=topics)

In [43]:
token_df

,topics,abajo,abandonara,abandone,abastecimiento,abatida,abierta,abierto,abismal,abismalesen,...,woyecheszen,xx,yaguarete,york,ypf,yuanes,zamora,zona,zonaprop,zonas
doc1,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc2,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc3,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc4,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc5,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
doc68,exchange,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
doc69,exchange,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc70,exchange,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc71,exchange,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0


In [44]:
token_df.to_pickle('tokens/topic_tokens_der.pkl')

#### CV tokens

In [45]:
pickle.dump(vectorizer, open( "tokens/cv_der.pkl", "wb" ))